# Building a database for crime reports

In this guided project, we will build a database for storing data related with crimes that occurred in Boston.

The goal of this  project is 
- create a database <font color='blue'>crimes_db</font>.
- create a table – <font color='blue'>boston_crimes</font> .
- create the <font color='blue'>readonly</font> group of users, which have permission to read data only.
- create the <font color='blue'>readwrite</font> group of users, which have permission to read data and alter data but not to delete tables.

The following diagram illustrates a high level overview of what we want to achieve:

![Image of Yaktocat](https://i.imgur.com/DCiYoMk.png)

## Creating the database and the schema

We will start by creating a database for storing our crime data as well as a schema for containing the tables.

Since the <font color='blue'>crime_db</font> does not exist yet, we can create it by connecting to the <font color='blue'>dq</font> database.

In [1]:
import psycopg2
conn = psycopg2.connect(dbname='dq', user='dq')
cur = conn.cursor()
conn.autocommit = True

cur.execute('CREATE DATABASE crime_db;')
conn.autocommit = False
conn.close()

conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
cur.execute('CREATE SCHEMA crimes;')


## Obtaining the Column Names and Sample

We now have a database and a schema — we are ready to start creating tables. Before we do that, let's gather some data about our crime dataset so that we can more easily select the right datatypes to use in our table.

Let's start by reading the column names from the <font color='blue'>boston.csv</font> file as well as the first row. In this way we will have them at hand throughout this guided project so that we can easily take a look at them at any moment.

In [2]:
import csv
with open('boston.csv') as file:
    reader = csv.reader(file)
    boston_lst = list(reader)
    col_headers = boston_lst[0]
    first_row = boston_lst[1]
    print(col_headers)
    print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


## Creating a function for analyzing column values


Before we create a table for storing the crime data, we need to identify the proper datatypes for the columns. 
To help us with that, let's create a function — <font color='blue'>et_col_value_set()</font> — that given the name of a CSV file and a column index (starting a 0) that computes a  <font color='blue'>Python set</font> with all distinct values contained in that column.

In [3]:
def get_col_value_set(csv_file, col_index):
    import csv
    values = set()
    with open(csv_file, 'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return len(values)

In [4]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [5]:
descr_col =  get_col_value_set('boston.csv', 0)
max_len = 0
for i in range(len(col_headers)):
    current_col =  get_col_value_set('boston.csv', i)
    print(col_headers[i] + ':  ' + str(current_col))

incident_number:  298329
offense_code:  219
description:  239
date:  1177
day_of_the_week:  7
lat:  18177
long:  18177


## Creating the table



Let's create a table named <font color='blue'>boston_crimes()</font> inside the <font color='blue'>crimes</font> schema of the <font color='blue'>crime_db</font> database:

- Colunm <font color='blue'>'incident_number'</font> will have INT4, max_len of this column is 6, but it has values greater than 32768.  
- Colunm <font color='blue'>'offense_code'</font> will have INT2, max_len of this column is 4.  
- Colunms <font color='blue'>'description'</font> will have  TEXT type. 
-  Colunms <font color='blue'>'day_of_the_week'</font> will have  WEEKDAYS type, we'll create it now.  
- Colunm <font color='blue'>'date'</font>  will have  DATE type.
- Colunms <font color='blue'>'lat'</font> and <font color='blue'>'long'</font> will have FLOAT8 type. FLOAT4 allows 6 digits, but our values are 10 digits. 

In [6]:
cur.execute("CREATE TYPE weekdays AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');")

cur.execute('''CREATE TABLE crimes.boston_crimes (
               incident_number INT4 PRIMARY KEY,
               offense_code INT4,
               description TEXT,
               date DATE,
               day_of_the_week weekdays,
               lat FLOAT8,
               long FLOAT8);'''
           )

## Load the data into the table¶

We used the copy_expert to load the data as it is very fast and very succinct to use. 

In [7]:
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
cur.execute("SELECT * FROM crimes.boston_crimes")
print(len(cur.fetchall()))

298329


## Revoke public privileges


Now it is time to handle users. Our goal is to create the two user groups that we have learned about: 
- readonly 
- readwrite 

By following the least privilege principle, the first step in doing so is to make sure that there are no privileges inherited from the public group and on the public schema.


In [8]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

## Creating the read only and read-write group

We create a readonly group with NOLOGIN because it is a group and not a user. We grant the group the ability to connect to the crime_db and the ability to use the crimes schema.

When dealing with a read only group of users, we will provide SELECT privilege. We also add an extra line compared with what was asked. This extra line changes the way that privileges are given by default to the readonly group on new table that are created on the crimes schema. As we mentioned, by default not privileges are given. However we change is so that by default any user in the readonly group can issue select commands.

When dealing with a read-write group of users, we will provide SELECT, INSERT, UPDATE and DELETE privileges. As before we change the default privileges so that user in the readwrite group have these privileges if we ever create a new table on the crimes schema.

In [9]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("CREATE GROUP readwrite NOLOGIN;")

cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")

cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")

cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")


## Creating users

Let's wrap up the project by creating one user in each group. We will need to create each user and then assign them to each group. 

In [10]:
# cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
# cur.execute("GRANT readonly TO data_analyst;")

# cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
# cur.execute("GRANT readwrite TO data_scientist;")

## Test the database setup

Test the database setup using SQL queries.

In the pg_roles table we will check database related privileges and for that we will look at the following columns:

- rolname: The name of the user / group that the privilege refers to.
- rolsuper: Whether this user / group is a super user. It should be set to False on every user / group that we have created.
- rolcreaterole: Whether user / group can create users, groups or roles. It should be False on every user / group that we have created.
- rolcreatedb: Whether user / group can create databases. It should be False on every user / group that we have created.
- rolcanlogin: Whether user / group can login. It should be True on the users and False on the groups that we have created.

In the information_schema.table_privileges we will check privileges related to SQL queries on tables. We will list the privileges of each group that we have created.

In [11]:
conn.commit()
conn.close()
# made new connection to check
conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
# check users and groups
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)
# check privileges
    
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur:
    print(user)
conn.close()

('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)
('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')
